In [63]:
#importing initial library
import json
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [64]:
#parsing json in python
with open('/content/algoparams_from_ui.json') as f:
  data = json.load(f)
print(type(data))

<class 'dict'>


In [65]:
#data in json
data

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_name'

In [66]:
session_name = data['session_name']
s_des = data['session_description']
print(session_name)
print(s_des)

test
test


#Designing State Data

In [67]:
df = pd.read_csv(data['design_state_data']['session_info']['dataset'])
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [68]:
data['design_state_data']['target']

{'prediction_type': 'Regression',
 'target': 'petal_width',
 'type': 'regression',
 'partitioning': True}

In [69]:
#save for further training
y = data['design_state_data']['target']['target']
y_df = df[y]
y_df

0      0.2
1      0.2
2      0.2
3      0.2
4      0.2
      ... 
145    2.3
146    1.9
147    2.0
148    2.3
149    1.8
Name: petal_width, Length: 150, dtype: float64

feature handling

In [72]:
# Create scaler
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))

In [73]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [74]:
#Feature handling
X_df = pd.DataFrame() #features_dataset
feature = data['design_state_data']['feature_handling']
for i in feature:
  if(feature[i]['is_selected'] == True):
    X_df[i] = df[i]
    if(feature[i]['feature_variable_type'] == 'numerical'):
      #numerical_handling
      if(feature[i]['feature_details']['numerical_handling'] == 'Keep as regular numerical feature'):
        if(feature[i]['feature_details']['rescaling'] == 'rescaling'):
          X_df[i] = minmax_scale.fit_transform(X_df[i].values.reshape(-1, 1))
        #missing_values
        if(feature[i]['feature_details']['missing_values'] =='Impute'):
          #impute with
          if(feature[i]['feature_details']['impute_with'] == 'custom'):
            X_df[i] = X_df[i].fillna(feature[i]['feature_details']['impute_value'])
          elif(feature[i]['feature_details']['impute_with'] == 'mean'):
            X_df[i] = X_df[i].fillna(df[i].mean())
          elif(feature[i]['feature_details']['impute_with'] == 'mode'):
            X_df[i] = X_df[i].fillna(df[i].mode())
          elif(feature[i]['feature_details']['impute_with'] == 'median'):
            X_df[i] = X_df[i].fillna(df[i].median())
      print(i)
    elif(feature[i]['feature_variable_type'] == 'text'):
      if(feature[i]['feature_details']['text_handling'] == 'Tokenize and hash'):
        if(feature[i]['feature_details']['hash_columns'] == 0):
          X_df[i] = label_encoder.fit_transform(X_df[i])
          X_df[i] = X_df[i] + 1
        else:
          a = i + '_hashed'
          X_df[a] = label_encoder.fit_transform(X_df[i])
          X_df[i] = X_df[i] + 1
      print(i)
X_df['species'].info()

sepal_length
sepal_width
petal_length
petal_width
species
<class 'pandas.core.series.Series'>
RangeIndex: 150 entries, 0 to 149
Series name: species
Non-Null Count  Dtype
--------------  -----
150 non-null    int64
dtypes: int64(1)
memory usage: 1.3 KB


In [75]:
from sklearn.preprocessing import RobustScaler

# Feature generation
feat_generation = data['design_state_data']['feature_generation']

# Initialize DataFrame to store new linear interaction features
new_linear_interaction_df = pd.DataFrame()

# For linear interactions
for interaction in feat_generation['linear_interactions']:
    feat_name = '_'.join(interaction)
    X_df[feat_name] = X_df[interaction[0]] * X_df[interaction[1]]
    new_linear_interaction_df[feat_name] = X_df[feat_name]

# Scale the new linear interaction features using different types of RobustScaler
for quantile_range in [(25, 75), (10, 90), (5, 95)]:  # Adjust quantile ranges as needed
    scaler = RobustScaler(quantile_range=quantile_range)
    scaled_linear_interaction_features = scaler.fit_transform(new_linear_interaction_df)
    new_linear_interaction_df_scaled = pd.DataFrame(scaled_linear_interaction_features, columns=new_linear_interaction_df.columns)
    X_df[new_linear_interaction_df_scaled.columns] = new_linear_interaction_df_scaled

# For polynomial interactions
for interaction in feat_generation['polynomial_interactions']:
    num_feature, denom_feature = interaction.split('/')

    # Convert the columns to numeric data types
    X_df[num_feature] = pd.to_numeric(X_df[num_feature], errors='coerce')
    X_df[denom_feature] = pd.to_numeric(X_df[denom_feature], errors='coerce')

    # Perform division and create new feature
    new_feature_name = interaction.replace('/', '_divided_by_')
    X_df[new_feature_name] = X_df[num_feature] / X_df[denom_feature]

# For explicit pairwise interactions
for interaction in feat_generation['explicit_pairwise_interactions']:
    feature1, feature2 = interaction.split('/')
    new_feature_name = feature1 + '_divided_by_' + feature2
    X_df[new_feature_name] = X_df[feature1] / X_df[feature2]


# Print the DataFrame information
X_df


,sepal_length,sepal_width,petal_length,petal_width,species,petal_length_sepal_width,petal_length_divided_by_sepal_width,petal_width_divided_by_species,sepal_width_divided_by_sepal_length,petal_width_divided_by_sepal_length
0,5.1,3.5,1.4,0.2,1,-0.498620,0.400000,0.200000,0.686275,0.039216
1,4.9,3.0,1.4,0.2,1,-0.546334,0.466667,0.200000,0.612245,0.040816
2,4.7,3.2,1.3,0.2,1,-0.549061,0.406250,0.200000,0.680851,0.042553
3,4.6,3.1,1.5,0.2,1,-0.515661,0.483871,0.200000,0.673913,0.043478
4,5.0,3.6,1.4,0.2,1,-0.489077,0.388889,0.200000,0.720000,0.040000
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,3,0.230735,1.733333,0.766667,0.447761,0.343284
146,6.3,2.5,5.0,1.9,3,0.019427,2.000000,0.633333,0.396825,0.301587
147,6.5,3.0,5.2,2.0,3,0.230735,1.733333,0.666667,0.461538,0.307692
148,6.2,3.4,5.4,2.3,3,0.418868,1.588235,0.766667,0.548387,0.370968


In [77]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
feat_red = data['design_state_data']['feature_reduction']
# Assume 'y' contains continuous values
# Convert 'y' into categorical labels using binning
y = pd.cut(y_df, bins=3, labels=['Low', 'Medium', 'High'])

# Initialize RandomForestClassifier
clf = RandomForestClassifier(n_estimators=int(feat_red['num_of_trees']), max_depth=int(feat_red['depth_of_trees']), random_state=42)

# Perform feature selection
selector = SelectFromModel(clf, max_features=int(feat_red['num_of_features_to_keep']))
X_reduced = selector.fit_transform(X_df, y)

# Get selected feature names
selected_features = X_df.columns[selector.get_support()]

# Create reduced DataFrame
X_reduced_df = pd.DataFrame(X_reduced, columns=selected_features)

# Print the reduced DataFrame
X_reduced_df


,petal_length,petal_width,species,petal_width_divided_by_sepal_length
0,1.4,0.2,1.0,0.039216
1,1.4,0.2,1.0,0.040816
2,1.3,0.2,1.0,0.042553
3,1.5,0.2,1.0,0.043478
4,1.4,0.2,1.0,0.040000
...,...,...,...,...
145,5.2,2.3,3.0,0.343284
146,5.0,1.9,3.0,0.301587
147,5.2,2.0,3.0,0.307692
148,5.4,2.3,3.0,0.370968


In [80]:
#algorithms
algorithms= data['design_state_data']['algorithms']
for i in algorithms:
  if(algorithms[i]['is_selected'] == True):
    print(algorithms[i]['model_name'])
    # Using dictionary comprehension
    sliced_params = {key: value for key, value in list(algorithms[i].items())[2:]}
    print(sliced_params)


Random Forest Regressor
{'min_trees': 10, 'max_trees': 20, 'feature_sampling_statergy': 'Default', 'min_depth': 20, 'max_depth': 25, 'min_samples_per_leaf_min_value': 5, 'min_samples_per_leaf_max_value': 10, 'parallelism': 0}


In [81]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GBTClassifier': GradientBoostingClassifier(),
    'GBTRegressor': GradientBoostingRegressor(),
    'LinearRegression': LinearRegression(),
    'LogisticRegression': LogisticRegression(),
    'RidgeRegression': Ridge(),
    'LassoRegression': Lasso(),
    'ElasticNetRegression': ElasticNet(),
    'xg_boost': xgb.XGBClassifier(),  # For classification tasks
    'xg_boost_regressor': xgb.XGBRegressor(),  # For regression tasks
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'SVM': SVC(),
    'SGD': SGDRegressor(),
    'KNN': KNeighborsClassifier(),
    'extra_random_trees': ExtraTreesRegressor(),
    'neural_network': MLPClassifier()
}


In [ ]:
from sklearn.model_selection import GridSearchCV


hyperparameters = data['design_state_data']['hyperparameters']
hyperparameters_tune = {key: value for key, value in list(hyperparameters.items())[2:]}
algorithms = data['design_state_data']['algorithms']
scores = []

for i in algorithms:
    if algorithms[i]['is_selected']:
        # Using dictionary comprehension
        sliced_params = {key: value for key, value in list(algorithms[i].items())[2:]}

        # Create GridSearchCV with the appropriate estimator
        clf = GridSearchCV(models[i], sliced_params, cv=5)  # Assuming 5-fold cross-validation

        # Fit the model
        clf.fit(X_reduced_df, y_df)

        # Append results
        scores.append({
            'model': i,
            'best_score': clf.best_score_,
            'best_params': clf.best_params_
        })

        print(i)
        print(sliced_params)
